# GUI - Nine Men Morris

In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

## Definitionen:
- ring = einer der 3 Quadrate (0-2)
- cell = ein Punkt auf einem Ring (0-7)
- position = (ring, cell)
- board = Array aus 3 Arrays, die angeben, ob, und wenn ja welcher Stein dort sitzt
- remaining = \[noch nicht gesetzte Steine Spieler 1, noch nicht gesetzte Steine Spieler 2\]
- pieces = \[remaining, board\]

In [ ]:
%run ./Muehle_Logic.ipynb
%run ./Muehle_Utilities.ipynb
import ipycanvas
from ipycanvas import Canvas, MultiCanvas

## Konstanten

* BOARD_SIZE = Größe des Spielfeldes in Pixeln
* DOT_RADIUS = Radius der schwarzen kleinen Punkte, die mögliche Positionen markieren (in Abhängigkeit von der Spielfeldgröße)
* PIECE_RADIUS = Radius der Spielsteine
* COLOUR = Farben der [dots, pieces_player_1, pieces_player_2]
* PADDING = relativer Abstand des äüßersten Quadrats zum Spielfeldrand
* DISTANCE = relativer Abstand zwischen den Quadraten des Spielfelds

In [ ]:
BOARD_SIZE   = 400
DOT_RADIUS   = BOARD_SIZE*0.025
PIECE_RADIUS = BOARD_SIZE*0.04
COLOUR       = ['black', 'white', 'sienna']
COLOUR_HINT  = 'green'
PADDING      = 0.05
DISTANCE     = 0.15
TRANSPARENCY_DEFAULT = 1.0
TRANSPARENCY_HINT    = 0.5

## Mögliche Positionen

<img src="board_positions.png" alt="Mögliche Positionen" width="800"/>

In [ ]:
#Reihe von oben nach unten (0-6)
def row(number):
    return PADDING + DISTANCE * number
#Spalte von links nach rechts (0-6)
def col(number):
    return row(number)

In [ ]:
positions = [([col(0), row(0)], [col(3), row(0)], [col(6), row(0)], [col(6), row(3)], [col(6), row(6)], [col(3), row(6)], [col(0), row(6)], [col(0), row(3)]), #ring 0
             ([col(1), row(1)], [col(3), row(1)], [col(5), row(1)], [col(5), row(3)], [col(5), row(5)], [col(3), row(5)], [col(1), row(5)], [col(1), row(3)]), #ring 1
             ([col(2), row(2)], [col(3), row(2)], [col(4), row(2)], [col(4), row(3)], [col(4), row(4)], [col(3), row(4)], [col(2), row(4)], [col(2), row(3)])] #ring 2

wmd_dic = {
    "a1":positions[0][6],
    "a4":positions[0][7],
    "a7":positions[0][0],
    "b2":positions[1][6],
    "b4":positions[1][7],
    "b6":positions[1][0],
    "c3":positions[2][6],
    "c4":positions[2][7],
    "c5":positions[2][0],
    "d1":positions[0][5],
    "d2":positions[1][5],
    "d3":positions[2][5],
    "d5":positions[2][1],
    "d6":positions[1][1],
    "d7":positions[0][1],
    "e3":positions[2][4],
    "e4":positions[2][3],
    "e5":positions[2][2],
    "f2":positions[1][4],
    "f4":positions[1][3],
    "f6":positions[1][2],
    "g1":positions[0][4],
    "g4":positions[0][3],
    "g7":positions[0][2]
}

## Initialisierung Canvas

**Aufbau Leinwand:**

board\[Hintergrund, Linien, Steine]

In [ ]:
#board[Hintergrund, Linien, Steine]
board = MultiCanvas(3, width = BOARD_SIZE, height = BOARD_SIZE)

# Hintergrund
board[0].fill_style = '#ffffcc'
board[0].fill_rect(0, 0, BOARD_SIZE)

# Strichstärke
board[1].line_width = 5

# Quadrate
board[1].stroke_rect(BOARD_SIZE*col(0), BOARD_SIZE*row(0), BOARD_SIZE*(1-row(0)-col(0))) #ring 0
board[1].stroke_rect(BOARD_SIZE*col(1), BOARD_SIZE*row(1), BOARD_SIZE*(1-row(1)-col(1))) #ring 1
board[1].stroke_rect(BOARD_SIZE*col(2), BOARD_SIZE*row(2), BOARD_SIZE*(1-row(2)-col(2))) #ring 2

# Mittelinien
board[1].begin_path()
board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(0)) #oben
board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(2))
board[1].move_to(BOARD_SIZE*col(6), BOARD_SIZE*row(3)) #rechts
board[1].line_to(BOARD_SIZE*col(4), BOARD_SIZE*row(3))
board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(6)) #unten
board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(4))
board[1].move_to(BOARD_SIZE*col(0), BOARD_SIZE*row(3)) #links
board[1].line_to(BOARD_SIZE*col(2), BOARD_SIZE*row(3))
board[1].stroke()

# Punkte (außen, mitte, innen)
for ring in positions:
    for x,y in ring:
        board[1].fill_arc(BOARD_SIZE*x, BOARD_SIZE*y, DOT_RADIUS, 0, 360)

## Status des Spielfelds
* 0 = kein Stein
* 1 = weißer Stein
* 2 = brauner Stein

In [ ]:
class status():
    pieces = [[9,9],[          # Anzahl zu setzender Steine (Spieler_1 (Weiß), Spieler_2 (Braun))
            [0,0,0,0,0,0,0,0], # ring 0
            [0,0,0,0,0,0,0,0], # ring 1
            [0,0,0,0,0,0,0,0]  # ring 2
      ]]
    current_player = 1
    mills = set()
    selected_dot_old = (None, None)
    selected_dot_new = (None, None)
    move = False
    game_over = False

## Spielsteine anzeigen

In [ ]:
def draw_piece(ring, cell, player):
    #Hintergrund
    board[2].fill_style = COLOUR[player]
    board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)
    #Ringe
    if player == 1:
        board[2].stroke_style = 'silver' 
    else:
        board[2].stroke_style = 'chocolate'
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS,     0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.7, 0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.3, 0, 360)

In [ ]:
def update_board(status):
    with ipycanvas.hold_canvas(board):
        board[2].clear()
        board[2].global_alpha = TRANSPARENCY_DEFAULT
        for ring in range(3):
            for cell in range(8):
                player = status.pieces[1][ring][cell]
                if player in [1, 2]: draw_piece(ring, cell, player)
        board[2].fill_style = 'black'
        board[2].font = '18px serif'
        board[2].fill_text('Spieler '+ str(status.current_player), BOARD_SIZE*0.4, BOARD_SIZE*0.45)
        board[2].fill_text('ist dran.', BOARD_SIZE*0.4, BOARD_SIZE*0.55)
    return board

In [ ]:
def on_dot(pos, x, y):
    return (pos[0]*BOARD_SIZE-PIECE_RADIUS/2 < x < pos[0]*BOARD_SIZE+PIECE_RADIUS/2 and pos[1]*BOARD_SIZE-PIECE_RADIUS/2 < y < pos[1]*BOARD_SIZE+PIECE_RADIUS/2)

In [ ]:
def show_possible_clickable_positions(possible_positions, colour = COLOUR_HINT):
    for position in possible_positions:
        ring, cell = position
        board[2].fill_style = colour
        board[2].global_alpha = TRANSPARENCY_HINT
        board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)
    return

In [ ]:
# def mouse_down(x_px, y_px):
#     global state
#     with output:
#         if not state.game_over:
#             x = math.floor(x_px / CELL_SIZE)
#             y = math.floor(y_px / CELL_SIZE)
#             try:
#                 state = make_move(state, (x, y))
#             except InvalidMoveException:
#                 print('Invalid Move')
#             update_output(state)
#             try:
#                 next_move(state)
#             except KeyboardInterrupt:
#                 pass
#
# canvas[1].on_mouse_down(mouse_down)

In [ ]:
def handle_mouse_down(x, y):
    global status
    if not status.game_over:
        for ring in positions:
            for cell in ring:
                if on_dot(cell, x, y):
                    status.selected_dot_old = status.selected_dot_new
                    status.selected_dot_new = (positions.index(ring), ring.index(cell))
                    try:
                        make_move(positions.index(ring), ring.index(cell))
                    except InvalidMoveException:
                        print('Ungültiger Zug')
                    update_board(status)
board[2].on_mouse_down(handle_mouse_down)

In [ ]:
# def make_move(state, pos):
#     if pos not in state.frontier:
#         print(pos, "not in Frontier")
#         raise InvalidMoveException
#     
#     state = copy.deepcopy(state)
#     disks_flipped = False
#     board = state.board.tolist()
#     for direction in directions:
#         if can_flip_in_dir(board, pos, direction, state.turn):
#             disks_flipped = True
#             flip_in_dir(state, pos, direction, state.turn)
# 
#     if disks_flipped:
#         state.num_pieces += 1
#         state.board[pos] = state.turn
#         state.last_move = pos
#         update_frontier(state, pos)
#         state.turn = -state.turn
#         state.possible_moves = get_possible_moves(state, state.turn)
#         if len(state.possible_moves) == 0:
#             state.turn = -state.turn
#             state.possible_moves = get_possible_moves(state, state.turn)
#             if len(state.possible_moves) == 0:
#                 state.game_over = True
#                 return state
#     else:
#         raise InvalidMoveException()
#     return state

In [ ]:
# def set_stone(x, y):
#     global status
#     board[2].global_alpha = TRANSPARENCY_DEFAULT
#     board[2].fill_style = 'red'
#     selected_ring, selected_cell = status.selected_dot
#     possible_positions = next_positions(status.pieces, status.current_player, selected_ring, selected_cell)
#     
#     
#     board[2].fill_text(possible_positions, 100, 100)
#     board[2].fill_text((selected_ring, selected_cell), 300, 300)
#     if not status.game_over:
#         try:
#             for ring in positions:
#                 for cell in ring:
#                     if on_dot(cell, x, y):
#                         board[2].fill_text((positions.index(ring), ring.index(cell)), 350, 350)
#                         if (positions.index(ring), ring.index(cell)) in possible_positions:
#                             board[2].fill_text('Test 2', 50, 350)
#                             status.pieces[1][positions.index(ring)][ring.index(cell)] = status.current_player
#                             status.pieces[1][selected_ring][selected_cell] = 0
#             board[2].on_mouse_down(handle_mouse_down) # normale Reaktion auf mouse down
#         except InvalidMoveException:
#             print('Ungültiger Zug')

In [ ]:
def make_move(ring, cell):
    global status
    phase = get_player_phase(status.pieces, status.current_player)
    possible_positions = next_positions(status.pieces, status.current_player, ring, cell)
    if phase == 1:
        if (ring, cell) in possible_positions: # Das geklickte Feld muss frei sein
            status.pieces[1][ring][cell] = status.current_player # Das geklickte Feld wird zum eigenen Feld
            status.pieces[0][status.current_player-1] = status.pieces[0][status.current_player-1]-1 # Die eigenen Steine in remaining um 1 reduzieren
            
            # ToDo: Mühle handeln
        else:
            raise InvalidMoveException()
    elif phase == 2:
        if status.move == False:
            show_possible_clickable_positions(possible_positions)
            board[2].fill_text('Switch to True', 300, 350)
            status.move = True
        elif status.move == True:
            board[2].fill_text('Test 0', 50, 350)
            old_ring, old_cell = status.selected_dot_old
            new_ring, new_cell = status.selected_dot_new
            possible_positions = next_positions(status.pieces, status.current_player, old_ring, old_cell)
            board[2].fill_text('Test 1', 50, 350)
            if (new_ring, new_cell) in possible_positions:
                board[2].fill_text('Test 2', 100, 350)
                status.pieces[1][new_ring][new_cell] = status.current_player
                status.pieces[1][old_ring][old_cell] = 0
                board[2].fill_text('Switch to False', 300, 350)
                status.move = False
                update_board(status)
            else:
                raise InvalidMoveException()
        # ToDo: Mühle handeln
    elif phase == 3:
        show_possible_clickable_positions(possible_positions)
        #board[2].on_mouse_down(set_stone)
        # ToDo: Mühle handeln
    else:
        raise InvalidMoveException()
    
    
    status.current_player = opponent(status.current_player) # Spielerwechsel
    if finish(status.pieces) != 0: # Prüfen, ob das Spiel zu Ende ist
        status.game_over = True
    board[2].fill_text('Test 2', 50, 350)
    #update_board(status)
    return status

In [ ]:
update_board(status)

In [ ]:
status.selected_dot_old

In [ ]:
status.move

In [ ]:
status.move = False

In [ ]:
status.pieces

In [ ]:
next_positions(status.pieces, status.current_player, 0, 3)